In [ ]:
!pip install -U langchain langchain_core langchain_openai pdfkit gradio wkhtmltopdf

In [2]:
import gradio as gr
import pandas as pd
import jinja2
import pdfkit
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_core.runnables import RunnableSequence, RunnablePassthrough, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import IpexLLM


In [3]:
# Step 1: Load CSV database
def load_csv(file):
    df = pd.read_csv(file.name)
    return df

# Step 1.1
def convert_binary_columns(df):
    for col in df.columns:
        if set(df[col].dropna().unique()) <= {0, 1}:  # Check if column is binary (only 0s and 1s)
            df[col] = df[col].map({1: 'Yes', 0: 'No'})  # Convert to Yes/No
    return df

#Step 1.2
def generate_data_summaries(df):
    # Numerical summary (after binary conversion)
    numerical_summary = df.describe().to_string()

    # Categorical summary
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    categorical_summary = {col: df[col].value_counts().to_string() for col in categorical_columns}
    #print(categorical_summary)
    
    return numerical_summary, categorical_summary

In [4]:
# Step 2: Set up LLM model (example using OpenAI API template)

llm = OpenAI(
            api_key="EMPTY",
            base_url="http://localhost:8000/v1",
            model="/home/dellubuntu/mylab/vllm/model_files/",
            max_tokens="2048"
        )

prompt_template = PromptTemplate.from_template(
    """You are an expert analyst. Given the following user question, numerical summary and categorical  summary - generate a detailed report.\
    Your report should include insights, detailed and elaborate analysis supported by analytical evidence, recommendations and a closing summary.\
    Make sure the response is focused and avoid unnecessary repetition. Your response should be exclusively provided in English language.
    Your responses should be formatted in HTML document following the rules as listed below:
    - <h2> tags For headers and <h4> tags for sub-headers
    - <ul> tags for unordered listings and <ol> for ordered listings
    - <b> tag for bolding of text
    - Follow all the other HTML syntax

    Question: {user_prompt}
    Numerical Summary: {numerical_summary}
    Categorical Summary: {categorical_summary}
    
    Answer: 

"""
)

# Step 3: Create an LLM Chain for report generation
report_chain = prompt_template | llm | StrOutputParser()

In [5]:
# Step 4: Function to generate report from the DataFrame
def generate_report_from_dataframe(user_prompt, file):
# def generate_report_from_dataframe(user_prompt, df):
    # Load the CSV filepath
    df = pd.read_csv(file.name)
    
    # Step to convert binary columns to Yes/No
    df = convert_binary_columns(df)

    # Generate summaries
    numerical_summary, categorical_summary = generate_data_summaries(df)
    
    # Generate the report using the LLM
    report = report_chain.invoke({"user_prompt": user_prompt,"numerical_summary": numerical_summary,
        "categorical_summary": "\n".join(f"{key}:\n{value}" for key, value in categorical_summary.items())})
    return report

In [6]:
# Step 5: Function to convert html report output to pdf 
def convert_html_to_pdf(
    html_string: str,
    template_folder: str = "./templates",
    base_html_file: str = "base.html",
    output_file: str = "generate_pdf.pdf",
    output_folder: str = "./generated_pdf_files",
) -> str:

    """
    Asynchronously converts an HTML file to a PDF file.

    This function takes the path of an HTML template file and generates a PDF file from it. The HTML file is expected to be located in the specified 'template_folder'. The generated PDF is saved with the name provided in 'output_file' and is stored in the specified 'output_folder'.

    Parameters:
    template_folder (str, optional): The path to the folder containing the HTML template. Defaults to "./templates".
    base_html_file (str, optional): The name of the base HTML file within the template folder. Defaults to "base.html".
    output_file (str, optional): The name of the output PDF file. Defaults to "generate_pdf.pdf".
    output_folder (str, optional): The path to the folder where the generated PDF file will be saved. Defaults to "../generated_pdf_files".

    Returns:
    str: The path to the generated PDF file.

    Note:
    - This function is asynchronous and should be awaited upon calling.
    - Ensure that the specified folders and files exist and are accessible.
    - The function might raise exceptions related to file reading/writing or PDF generation which should be handled appropriately.
    """

    if output_folder.endswith("/"):
        raise ValueError("Wrong output folder name, should not end with '/'")
    else:
        pdf_file_name = f"{output_folder}/{output_file}"

    try:
        template_loader = jinja2.FileSystemLoader(template_folder)
        template_env = jinja2.Environment(loader=template_loader)

        basic_template = template_env.get_template(base_html_file)

        output_html_code = basic_template.render()
        # print(output_html_code)

        # render content, this if for once we have AI generated response
        output_html_code = basic_template.render(
            ai_generated_content=html_string
        )

        options = {
            'page-size': 'A4',
            'margin-top': '0.75in',
            'margin-bottom': '0.75in',
            'margin-right': '0.55in',
            'margin-left': '0.55in',
            'encoding': "UTF-8",
            'footer-right': '[page] of [topage]',
            'footer-font-size': "9",
            'custom-header': [
                ('Accept-Encoding', 'gzip')
            ],
            'enable-local-file-access': False,
            'no-outline': None,
            'enable-local-file-access': False,
            'no-outline': None
        }

        config = pdfkit.configuration(wkhtmltopdf="C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe")

        pdfkit.from_string(
            input=output_html_code,
            output_path=pdf_file_name,
            options=options,
            configuration=config
        )

    except Exception as e:
        # good to log this exception instead
        print(e)
        return ""

    return f"PDF generated successfully at {pdf_file_name}"

In [ ]:

with gr.Blocks(theme=gr.Theme.from_hub('HaleyCH/HaleyCH_Theme'), css=".column-form .wrap {flex-direction: column;}") as app:
    with gr.Row():
        gr.Markdown("""<h1><center>Report Generator</center>""")

    with gr.Row():
        with gr.Column(visible=True, min_width=350, scale=0) as sidebar:
            with gr.Row():
                # CSV file uploader
                csv_file_input = gr.File(label="Upload your CSV file", file_types=[".csv"])


            with gr.Row():
                # Textbox for user prompt
                user_prompt_input = gr.Textbox(
                    label="Enter focus area for analysis",
                    placeholder="e.g., What are the 3 main reasons for customer churn?\nPress `Enter` to generate report",
                    lines=3
                )

            with gr.Row():
                generate_pdf = gr.Button(
                    value="Press to convert report to PDF"
                )
        
        with gr.Column() as main:
            with gr.Row():
                # Output area for the generated report
                output_html = gr.HTML(label="Generated Report")
    
    # Set up the action to happen when the button is clicked
    user_prompt_input.submit(
        fn=generate_report_from_dataframe,
        inputs=[user_prompt_input, csv_file_input],
        outputs=output_html
        #outputs=textbox
    )

    # Convert HTML to PDF and display info message
    generate_pdf.click(
        fn=lambda html_content: (
        convert_html_to_pdf(html_content),
        gr.Info(convert_html_to_pdf(html_content))
        ),
        inputs=output_html,
        outputs=None
    )    

# Launch the Gradio app
app.launch(share=True, debug=True)